<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/main/Code/ReHealthCo_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReHealthCo EDA-Prozess

## Vorbereitung 

###CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books (Ziel Reproduzierbarkeit)

In [ ]:
# Löschen aller Variablen
%reset -f

Installtion der Package ydata_profiling

In [ ]:
!pip install ydata_profiling

### Importieren der notwendigen Libraries: sqlite3, csv, pandas

In [ ]:
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
from pandas.util import hash_pandas_object
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt

Erstellung einer virtuellen Kopie des GitHub Repositorium DMA2023TeamD in Colab

In [ ]:
!git clone https://github.com/Fuenfgeld/DMA2023TeamD.git

In [ ]:
# Wechsel in den Unterordner csv_data des Github-Projekt-Ordners
%cd /content/DMA2023TeamD/csv_data

In [ ]:
# Auflistung aller Ordnerinhalte
!ls

### Anlegen des rhc-DataFrames *rhc_df* und Einlesen des anonymisierten csv-Files von Github

In [ ]:
# Pfad-Abkürzung zum Speicherort der anonymisierten csv-Tabelle
csv_path_anonymrhc = "/content/DMA2023TeamD/csv_data"

In [ ]:
# Einlesen der anonymisierten Tabelle als Pandas DataFrame
rhc_df = pd.read_csv(f"{csv_path_anonymrhc}/anonym_rhcpat.csv")

In [ ]:
# Anzeige des DF-Inhalts
rhc_df

##Auswertungen auf ***rhc_df***

In [ ]:
# Anzeige der Kopfspalten des DF
print(rhc_df.columns.tolist())

###Diagnosen/Patient (0,1, 2 und >3 Diagnosen)

In [ ]:
# Diagnosen/Patient
rhc_df[["RHC_PSEUDONYM", "DESC_CON"]].groupby(["RHC_PSEUDONYM"]).size()

#### Anlegen der Spalte  *diag_anz_df* (Diagnosen pro Patient)

In [ ]:
# Anzeige Diagnosen/Patient, absteigend sortiert
diag_anz_df = rhc_df[["RHC_PSEUDONYM", "DESC_CON"]].groupby(["RHC_PSEUDONYM"]).count().sort_values("DESC_CON", ascending=False) 
diag_anz_df.rename(columns={"DESC_CON": "DESC_CON_ANZ"}, inplace=True)
diag_anz_df

In [ ]:
# Spalte Diagnosen/Patient an das gesamte DataFrame hängen
rhc_diagn_df = pd.DataFrame.merge(rhc_df, diag_anz_df, left_on=["RHC_PSEUDONYM"], right_on=["RHC_PSEUDONYM"], how="left")
rhc_diagn_df

####Anlegen der Spalte *DESC_CON_SORT* (Value o,1,2, >3)

In [ ]:
# Spalte DESC_CON_ANZ verdoppeln
rhc_diagn_df.loc[:, 'DESC_CON_SORT'] = rhc_diagn_df.loc[:, 'DESC_CON_ANZ']
# Anzeige der Kopfspalten des DF
print(rhc_diagn_df.columns.tolist())

In [ ]:
# Ersetzen aller Werte bei Patienten > 2 Diagnosen mit einer 3
rhc_diagn_df['DESC_CON_SORT'].values[rhc_diagn_df['DESC_CON_SORT'] > 2] = 3
rhc_diagn_df

##Arbeit auf *rhc_diagn_df*

### Spalte *HEALTHCARE_EXPENSES_UNCOVERED*

In [ ]:
# neue Spalte HEALTHCARE_EXPENSES uncovered
rhc_diagn_df["HEALTHCARE_EXPENSES_UNCOVERED"] = rhc_diagn_df["HEALTHCARE_EXPENSES_PAT"]-rhc_diagn_df["HEALTHCARE_COVERAGE_PAT"]
rhc_diagn_df

###Spalte *VERH_HEALTHCARE_EXPENSES_COV_UNCOV* 
Verhältnis Ausgaben abgedeckt/nicht abgedeckt

In [ ]:
# neue Spalte Verhältnis Ausgaben abgedeckt/nicht abgedeckt mit Spaltennahmen VERH_HEALTHCARE_EXPENSES_COV_UNCOV
rhc_diagn_df["VERH_HEALTHCARE_EXPENSES_COV_UNCOV"] = rhc_diagn_df["HEALTHCARE_COVERAGE_PAT"]/rhc_diagn_df["HEALTHCARE_EXPENSES_PAT"]
rhc_diagn_df

## Patient pro Diagnose


### *diagn_enc_df*, Patient pro Diagnose mit Encounterclasse Wellness 

In [ ]:
# Anzeige DF Patienten/Diagnose, Erweiterung sowie Encountbegegnungen, absteigend sortiert nach Patienten/Diagnose
diagn_enc_df = rhc_df[["RHC_PSEUDONYM", "DESC_CON", "ENCOUNTERCLASS_ENC"]].groupby(["DESC_CON"]).count().sort_values("RHC_PSEUDONYM", ascending=False) 
diagn_enc_df

### *diagn_encnowell_df* Patient pro Diagnose ohne Encounterclasse wellness

In [ ]:
# Anzeige DF Patienten/DiagnoseErweiterung sowie Encountbegegnungen, absteigend sortiert nach Patienten/Diagnose
pat_anz_nowellrechnung_df = rhc_df.drop(rhc_df[ (rhc_df['ENCOUNTERCLASS_ENC'] == "wellness") ].index) # Löschen der Encounterbegegnung wellness 
diagn_encnowell_df = pat_anz_nowellrechnung_df[["RHC_PSEUDONYM", "DESC_CON", "ENCOUNTERCLASS_ENC"]].groupby(["DESC_CON"]).count().sort_values("RHC_PSEUDONYM", ascending=False) # Benennung des DF
diagn_encnowell_df.rename(columns={"ENCOUNTERCLASS_ENC": "ENCOUNTERCLASS_NOWELL"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Diagnose in ENCOUNTERCLASS_NOWELL
diagn_encnowell_df # Anzeige DF

##Encounterbegegnungen pro Patient

###*rhc_enc_anz_df*  Encounterbegegnungen/Patient

In [ ]:
# Anzeige Encounterbegegnungen/Patient, absteigend sortiert
rhc_enc_anz_df = rhc_diagn_df[["RHC_PSEUDONYM", "ENCOUNTERCLASS_ENC"]].groupby(["RHC_PSEUDONYM"]).count().sort_values("ENCOUNTERCLASS_ENC", ascending=False) 
rhc_enc_anz_df.rename(columns={"ENCOUNTERCLASS_ENC": "ENCOUNTERCLASS_ENC_ANZ"}, inplace=True)
rhc_enc_anz_df

## Erstellen *rhc_all_df* (rhc_diagn_df und rhc_enc_anz_df)

In [ ]:
# Spalte Encounterbegegnungen/Patient an das gesamte DataFrame hängen
rhc_all_df = pd.DataFrame.merge(rhc_diagn_df, rhc_enc_anz_df, left_on=["RHC_PSEUDONYM"], right_on=["RHC_PSEUDONYM"], how="left")
rhc_all_df

####Spalte *ENCOUNTERCLASS_ENC_ANZ*

####Spalte *ENCOUNTERCLASS_ENC_ANZ_SORT*

In [ ]:
# Ersetzen aller Werte bei Patienten > 2 Encounterbegegnungen mit einer 3 nach Verdopplung
rhc_all_df.loc[:, 'ENCOUNTERCLASS_ENC_ANZ_SORT'] = rhc_all_df.loc[:, 'ENCOUNTERCLASS_ENC_ANZ'] # Spalte ENCOUNTERCLASS_ENC_ANZ verdoppeln
rhc_all_df['ENCOUNTERCLASS_ENC_ANZ_SORT'].values[rhc_all_df['ENCOUNTERCLASS_ENC_ANZ_SORT'] > 2] = 3
rhc_all_df # Anzeige des Ergebnis

In [ ]:
# Löschen der Encounterbegegnung wellness 
rhc_all_nowell_df = rhc_all_df.drop(rhc_all_df[ (rhc_all_df['ENCOUNTERCLASS_ENC'] == "wellness")
                                                     ].index)
rhc_all_nowell_df.groupby(["ENCOUNTERCLASS_ENC"]).size() # Anzeige der Übersicht

In [ ]:
rhc_all_nowell_df # Anzeige des Ergebnis

In [ ]:
# Umschreibung aller string-werte in integer vor Gruppierung auf einzelnen Patienten
rhc_all_nowell_df.ENCOUNTERCLASS_ENC.replace(['ambulatory', 'inpatient', 'emergency', 'outpatient'], [0, 1, 2, 3], inplace=True)
rhc_all_nowell_df.DATENBANKART.replace(['asthma', 'covid', 'metabolic'], [0, 1, 2], inplace=True)
rhc_all_nowell_df.DESC_CON.replace(['COVID-19', 'Coronary Heart Disease', 'Pneumonia (disorder)', "Hypoxemia (disorder)", "Acute respiratory failure (disorder)", "Cardiac Arrest", "Atrial Fibrillation", "Stroke", "Sepsis caused by virus (disorder)", 
                                        "Acute deep venous thrombosis (disorder)", "Acute pulmonary embolism (disorder)", "Myocardial Infarction", "Septic shock (disorder)", "Acute respiratory distress syndrome (disorder)",
                                        "Injury of heart (disorder)", "Acquired coagulation disorder (disorder)", "Childhood asthma", "Injury of kidney (disorder)", "Heart failure (disorder)", "Bacterial infectious disease (disorder)", "Asthma"
                                        ], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], inplace=True)
rhc_all_nowell_df["DESC_CON"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Anzeige der Kopfspalten des DF
print(rhc_all_nowell_df.columns.tolist())

In [ ]:
# Gruppierung nach Patienten, diese sonst mehrfach gezählt werden
groupbypat_df = rhc_all_nowell_df[['RHC_PSEUDONYM', 'GENDER_PAT', 'HEALTHCARE_EXPENSES_PAT', 'HEALTHCARE_COVERAGE_PAT', 'DATENBANKART', 'LEBENDIG', 'AGE', 'CODE_CON', 'DESC_CON', 'CONDITIONDAUER', 'TOTALCOST_MED', 'ENCOUNTERCLASS_ENC', 'BEHANDLUNGSDAUER', 'DESC_CON_ANZ', 'DESC_CON_SORT', 'HEALTHCARE_EXPENSES_UNCOVERED', 'VERH_HEALTHCARE_EXPENSES_COV_UNCOV', 'ENCOUNTERCLASS_ENC_ANZ', 'ENCOUNTERCLASS_ENC_ANZ_SORT']
].groupby(["RHC_PSEUDONYM"]).mean().reset_index()
groupbypat_df

In [ ]:
# Profilreporting nach Groupierung auf den Patienten, da sonst Patienten mehrfach in der Auswertung auftreten und dies die Ergebnisse verzerrt
ProfileReport(groupbypat_df)

In [ ]:
anz_pat_dign_enc_df = diagn_encnowell_df.reset_index()
anz_pat_dign_enc_df.rename(columns={"RHC_PSEUDONYM": "PAT_ANZ", "DESC_CON": "DIAGNOSEN", "ENCOUNTERCLASS_NOWELL": "ENC_BEGEG_NOWELL"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Diagnose in ENCOUNTERCLASS_NOWELL
ProfileReport(anz_pat_dign_enc_df)

# Boxplot and Remove Outlier 
(Extremwerte rausnehmen)

In [ ]:
# Anzeige der Kopfspalten des DF
print(groupbypat_df.columns.tolist())

In [ ]:
sns.boxplot(x=groupbypat_df["HEALTHCARE_COVERAGE_PAT"])

In [ ]:
rhc_rein01_df = groupbypat_df[groupbypat_df["HEALTHCARE_COVERAGE_PAT"]<15000]
sns.boxplot(x=rhc_rein01_df["HEALTHCARE_COVERAGE_PAT"])

In [ ]:
sns.boxplot(x=rhc_rein01_df["CONDITIONDAUER"])

In [ ]:
rhc_rein02_df = rhc_rein01_df[rhc_rein01_df["CONDITIONDAUER"]<1000]
sns.boxplot(x=rhc_rein01_df["CONDITIONDAUER"])

In [ ]:
sns.boxplot(x=rhc_rein02_df["BEHANDLUNGSDAUER"])

In [ ]:
rhc_rein03_df = rhc_rein02_df[rhc_rein02_df["BEHANDLUNGSDAUER"]<200]
sns.boxplot(x=rhc_rein03_df["CONDITIONDAUER"])

In [ ]:
sns.boxplot(x=rhc_rein03_df["DESC_CON_ANZ"])

In [ ]:
rhc_rein_df = rhc_rein03_df[rhc_rein03_df["DESC_CON_ANZ"]<100]
sns.boxplot(x=rhc_rein_df["DESC_CON_ANZ"])

### um Extremwerten bereinigtes DF *rhc_rein_df*

In [ ]:
rhc_rein_df

### Vergleich der Zeilenanzahl der DFs vor und nach Extremwertbereinigung: nach Bereinigung Verlust von 2814 Zeilen obwohl nur in 4 Spalten Wertbereinigung vorgenommen wurde und nur offensichtliche Ausreisser rausgestrichen wurden

In [ ]:
# Anzeige aller Zeilen
rhc_rein_df.shape[0]

In [ ]:
# Anzeige aller Zeilen
groupbypat_df.shape[0]

In [ ]:
# distribution of GENDER attribute ohne Extremwerten
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["GENDER_PAT"]]))

In [ ]:
# distribution of GENDER attribute mit Extremwerten
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["GENDER_PAT"]]))

In [ ]:
# Verhältnis Diagnosen-Anzahl/Patient im bereinigten DF
sns.countplot(x="variable", hue= "value", orient="v", data=pd.melt(rhc_rein_df[["DESC_CON_ANZ"]]))

In [ ]:
# SJ 
# Verhältnis Diagnosen-Anzahl/Patient im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["DESC_CON_ANZ"]]))

In [ ]:
# Verhältnis Diagnosen-Anzahl/Patient im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["DESC_CON_ANZ"]]))

In [ ]:
# Verteilung Behandlungsdauer in Stunden im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["BEHANDLUNGSDAUER"]]))

In [ ]:
# Verteilung Behandlungsdauer in Stunden im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["BEHANDLUNGSDAUER"]]))

In [ ]:
# Verteilung Conditiondauer in Stunden im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["CONDITIONDAUER"]]))

In [ ]:
# Verteilung Conditiondauer in Stunden im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["CONDITIONDAUER"]]))

In [ ]:
# Verteilung der Diagnosenhäufigkeit/Patient im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["DESC_CON_ANZ"]]))

In [ ]:
# Verteilung der Diagnosenhäufigkeit/Patient im nicht DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["DESC_CON_ANZ"]]))

In [ ]:
# Verteilung der Erkrankungen im nicht bereinigten DF mit wellness
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_all_df[["DESC_CON"]]))

In [ ]:
# Verteilung der Erkrankungen im nicht bereinigten DF ohne wellness mit folgender Zuweisung: 
# ['COVID-19', 'Coronary Heart Disease', 'Pneumonia (disorder)', "Hypoxemia (disorder)", "Acute respiratory failure (disorder)", "Cardiac Arrest", "Atrial Fibrillation", "Stroke", "Sepsis caused by virus (disorder)", 
# "Acute deep venous thrombosis (disorder)", "Acute pulmonary embolism (disorder)", "Myocardial Infarction", "Septic shock (disorder)", "Acute respiratory distress syndrome (disorder)",
# "Injury of heart (disorder)", "Acquired coagulation disorder (disorder)", "Childhood asthma", "Injury of kidney (disorder)", "Heart failure (disorder)", "Bacterial infectious disease (disorder)", "Asthma"]
# = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_all_nowell_df[["DESC_CON"]]))

In [ ]:
# Verteilung Diagnosen/Patient
sns.countplot(x="variable", hue= "value", data=pd.melt(diagn_enc_df[["RHC_PSEUDONYM"]]))

In [ ]:
# Verteilung Diagnosen/Encounterbegenung
sns.countplot(x="variable", hue= "value", data=pd.melt(diagn_enc_df[["ENCOUNTERCLASS_ENC"]]))

In [ ]:
# Histogramm HEALTHCARE_EXPENSES_UNCOVERED im nicht bereinigten DF
groupbypat_df["HEALTHCARE_EXPENSES_UNCOVERED"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Histogramm HEALTHCARE_EXPENSES_UNCOVERED im bereinigten DF
rhc_rein_df["HEALTHCARE_EXPENSES_UNCOVERED"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Histogramm Verhätnis Kosten gedeckt/nicht abgedeckt im nicht bereinigten DF
groupbypat_df["VERH_HEALTHCARE_EXPENSES_COV_UNCOV"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Histogramm Verhätnis Kosten gedeckt/nicht abgedeckt im bereinigten DF
rhc_rein_df["VERH_HEALTHCARE_EXPENSES_COV_UNCOV"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Histogramm Alter im nicht bereinigten DF
groupbypat_df["AGE"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Histogramm Alter im bereinigten DF
rhc_rein_df["AGE"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Verteilung tot/lebend (1/0) nicht bereinigte DF
rhc_rein_df["LEBENDIG"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Verhältnis tot/lebend 0/1 im bereinigten DF in Farbe
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["LEBENDIG"]]))

In [ ]:
# Verhältnis tot/lebend 0/1 im nicht bereinigten DF in Farbe
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["LEBENDIG"]]))

In [ ]:
# Verhältnis Gesundheitskostenabeckung im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["HEALTHCARE_COVERAGE_PAT"]]))

In [ ]:
# Verhältnis Gesundheitskostenabeckung im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["HEALTHCARE_COVERAGE_PAT"]]))

In [ ]:
# Verhältnis Ursprungstabelle im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["DATENBANKART"]]))

In [ ]:
# Verhältnis Ursprungstabelle im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["DATENBANKART"]]))

In [ ]:
# Verhältnis Medikamentenkosten im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["TOTALCOST_MED"]]))

In [ ]:
# Verhältnis Medikamentenkosten im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["TOTALCOST_MED"]]))

In [ ]:
# Verhältnis Behandlungsdauer im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["BEHANDLUNGSDAUER"]]))

In [ ]:
# Verhältnis Behandlungsdauer im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["BEHANDLUNGSDAUER"]]))

In [ ]:
# Verhältnis Diagnosen-Anzahl/Patient 0/1/2/3(= >2) im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["DESC_CON_SORT"]]))

In [ ]:
# Verhältnis Diagnosen-Anzahl/Patient 0/1/2/3 (= >2) im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["DESC_CON_SORT"]]))

In [ ]:
# Verhältnis Encounterbegegnungen im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["ENCOUNTERCLASS_ENC_ANZ"]]))

In [ ]:
# Verhältnis Encounterbegegnungen im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["ENCOUNTERCLASS_ENC_ANZ"]]))

In [ ]:
# Verhältnis Encounterbegegnungen /1/2/3 (= >2) im bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(rhc_rein_df[["ENCOUNTERCLASS_ENC_ANZ_SORT"]]))

In [ ]:
# Verhältnis Encounterbegegnungen /1/2/3 (= >2) im nicht bereinigten DF
sns.countplot(x="variable", hue= "value", data=pd.melt(groupbypat_df[["ENCOUNTERCLASS_ENC_ANZ_SORT"]]))

# Multivariate Analysis: Analyzing Relationships Between Variables

In [ ]:
# Generating the correlating matrix
corr = rhc_all_nowell_df.corr()
plt.figure(figsize=(7, 5))
# Generating the correlation heat-map
sns.heatmap(corr, annot=True)

# Version Check

In [ ]:
#python Version
import sys
sys.version_info

In [ ]:
pd.__version__

In [ ]:
csv.__version__

In [ ]:
sns.__version__

In [ ]:
# Überprüfen aller Variablen
# %pip freeze

# Calculate the Checksum of the Data Set

In [ ]:
rhc_diagn_df_hashes = hash_pandas_object(rhc_diagn_df)
diag_anz_df_hashes = hash_pandas_object(diag_anz_df)
rhc_diagn_df_hashes = hash_pandas_object(rhc_diagn_df)
diagn_enc_df_hashes = hash_pandas_object(diagn_enc_df)
pat_anz_nowellrechnung_df_hashes = hash_pandas_object(pat_anz_nowellrechnung_df)
diagn_encnowell_df_hashes = hash_pandas_object(diagn_encnowell_df)
rhc_enc_anz_df_hashes = hash_pandas_object(rhc_enc_anz_df)
rhc_enc_anz_df_hashes = hash_pandas_object(rhc_enc_anz_df)
rhc_all_df_hashes = hash_pandas_object(rhc_all_df)
rhc_all_nowell_df_hashes = hash_pandas_object(rhc_all_nowell_df)
groupbypat_df_hashes = hash_pandas_object(groupbypat_df)
rhc_rein01_df_hashes = hash_pandas_object(rhc_rein01_df)
rhc_rein02_df_hashes = hash_pandas_object(rhc_rein02_df)
rhc_rein03_df_hashes = hash_pandas_object(rhc_rein03_df)
rhc_rein_df_hashes = hash_pandas_object(rhc_rein_df)
anz_pat_dign_enc_df_hashes = hash_pandas_object(anz_pat_dign_enc_df)

In [ ]:
Liste_rhcedaall_hashes = [rhc_diagn_df_hashes, diag_anz_df_hashes, rhc_diagn_df_hashes, diagn_enc_df_hashes, pat_anz_nowellrechnung_df_hashes, diagn_encnowell_df_hashes, rhc_enc_anz_df_hashes, rhc_enc_anz_df_hashes, rhc_all_df_hashes, rhc_all_nowell_df_hashes, groupbypat_df_hashes, rhc_rein01_df_hashes, rhc_rein02_df_hashes, rhc_rein03_df_hashes, rhc_rein_df_hashes, anz_pat_dign_enc_df_hashes]

In [ ]:
print(Liste_rhcedaall_hashes)

## Value should be 
2650178518257614002

-3795146034381248713

2650178518257614002

7831896322564588659

-840543143014388252

1693621233367357631

-5022892886560696056

8245114429953816038

-3815936067771996962

-7423060998095774579

-7611279281022461570

-6161144217749052061

6171608350576877069

2939134733002627004

-1250212140467252653

In [ ]:
print(rhc_diagn_df_hashes.sum())
print(diag_anz_df_hashes.sum())
print(rhc_diagn_df_hashes.sum())
print(diagn_enc_df_hashes.sum())
print(pat_anz_nowellrechnung_df_hashes.sum())
print(diagn_encnowell_df_hashes.sum())
print(rhc_enc_anz_df_hashes.sum())
print(rhc_all_df_hashes.sum())
print(rhc_all_nowell_df_hashes.sum())
print(groupbypat_df_hashes.sum())
print(rhc_rein01_df_hashes.sum())
print(rhc_rein02_df_hashes.sum())
print(rhc_rein03_df_hashes.sum())
print(rhc_rein_df_hashes.sum())
print(anz_pat_dign_enc_df_hashes.sum())